In [5]:
import librosa, glob, sys, json, pickle, os, random, linecache, statistics
from tqdm import tqdm
import pandas as pd, numpy as np
from IPython.core.display import display, HTML
from collections import Counter
display(HTML("<style>.container { width:100% !important; }</style>"))

In [9]:
def convert_json(file_type, accent):
    ref = open(f'{accent}/all.json', 'r')
    files = ref.readlines()
    files = [json.loads(file) for file in files]
    path_dict = [(file["audio_filepath"], file["pseudo_text"]) for file in files]
    path_dict = dict(path_dict)
    json_file = open(f"{accent}/manifests/{file_type}.json", 'r')
    samples = json_file.readlines()
    new_json_file = open(f"{accent}/{file_type}.json", 'w')
#     new_json_file = open(f"{speaker}/temp.json", 'w')
    new_json_samples = []
    for sample in samples:
        new_dict = json.loads(sample)
        new_dict['pseudo_text'] = path_dict[new_dict['audio_filepath']]
        json.dump(new_dict, new_json_file)
        new_json_file.write('\n')

In [10]:
accents = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
file_types = ["all", "test", "selection", "seed", "seed_plus_dev", "dev"]
for accent in accents:
    for file_type in file_types:
        if file_type in ['all', 'seed_plus_dev']: continue
        convert_json(file_type, accent)

In [12]:
for accent in accents:
    file_type = 'seed_plus_dev'
    seed_file = open(f"{accent}/seed.json", 'r')
    dev_file = open(f"{accent}/dev.json", 'r')
    seed_samples = seed_file.readlines()
    dev_samples = dev_file.readlines()
    dev_samples = dev_samples[:50]
    new_json_file = open(f"{accent}/{file_type}.json", 'w')
    new_json_samples = []
    samples = seed_samples+dev_samples
    random.shuffle(samples)
    for sample in samples:
        json.dump(json.loads(sample), new_json_file)
        new_json_file.write('\n')

In [13]:
# setting up random selections
accents = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
# random.seed(42)

def _write(manifests_path, items, file):
    with open(manifests_path+file, 'w') as f:
        for item in items:
            json.dump(item, f)
            f.write('\n')

In [ ]:
!wc -l */selection.json
!wc -l random/all_selection.json

In [14]:
global_files = []
manifests_path = 'random/'

for accent in accents:
	accent_json = open(accent+'/selection.json', 'r')
	files = accent_json.readlines()
	files = [json.loads(file) for file in files]
	random.shuffle(files)
	global_files.extend(files)
	print(len(global_files))

random.shuffle(global_files)
_write(manifests_path, global_files, 'all_selection.json')
def budget(budget_size):
    return int(5.4*budget_size)
# os.makedirs(output_dir, exist_ok=True)
ground_list = [json.loads(line.strip()) for line in open('random/all_selection.json')]

1350
6200
7200
8550
14100
15450
16450
17800
19850
33800
33800
33800
33800
33800


In [15]:
for budget_size in [100, 200, 250, 300, 400, 500, 600, 700, 800, 1000, 2000, 2500, 3500]:
    output_dir = 'random/{}'.format(budget_size)
    list_total_selection, list_total_count, list_total_duration = [], [], []
    list_accent_sample_count, list_accent_sample_duration = {}, {}
    for accent in accents:
        list_accent_sample_count[accent] = []
        list_accent_sample_duration[accent] = []
    
    for i in [1, 2, 3]:
        run = f'run_{i}'
        run_dir = os.path.join(output_dir, run)
        os.makedirs(run_dir, exist_ok=True)
        # for folder in ['train', 'output', 'plots']:
        #     os.makedirs(os.path.join(run_dir, folder))
        all_indices = list(range(len(ground_list)))
        random.seed(i)
        random.shuffle(all_indices)
        total_duration, index = 0, 0
        while total_duration + ground_list[all_indices[index]]['duration'] <= budget(budget_size):
            total_duration += ground_list[all_indices[index]]['duration']
            index += 1
        list_total_count.append(index)
        list_total_duration.append(total_duration)
        selected_indices = all_indices[:index]
        selected_list = [ground_list[j] for j in selected_indices]

#        train_list = selected_list + query_list
        train_list = selected_list
        
        for accent in accents:
            accent_sample_count, accent_sample_duration = 0, 0
            for item in selected_list:
                if item['audio_filepath'].split('/')[-4] == accent:
                    accent_sample_count += 1
                    accent_sample_duration += item['duration']
            list_accent_sample_count[accent].append(accent_sample_count)
            list_accent_sample_duration[accent].append(accent_sample_duration)
        list_total_selection.append(Counter([item['audio_filepath'].split('/')[-4] for item in selected_list]))
        
#        with open(base_dir + query_dir + f'train/error_model/{budget_size}/seed_{i}/train.json', 'w') as f:
#            for line in train_list:
#                f.write('{}\n'.format(json.dumps(line)))
        with open(f'{run_dir}/train.json', 'w') as f:
            for line in train_list:
                f.write('{}\n'.format(json.dumps(line)))
                
#        plots(dirs, run_dir, ground_features, ground_features_Y, query_features, test_features, selected_indices)
    
    stats = 'total selection : ' + ' '.join(map(str, list_total_count)) + ' -> {0:.2f}\n'.format(statistics.mean(list_total_count))
    stats += 'total selection duration: ' + ' '.join(map(str, list_total_duration)) + ' -> {0:.2f}\n'.format(statistics.mean(list_total_duration))
    for accent in accents:
        stats += '\naccent: {}\n'.format(accent)
        stats += 'accented selection: ' + ' '.join(map(str, list_accent_sample_count[accent])) + ' -> {0:.2f}\n'.format(statistics.mean(list_accent_sample_count[accent]))
        stats += 'accented duration: ' + ' '.join(map(str, list_accent_sample_duration[accent])) + ' -> {0:.2f}\n'.format(statistics.mean(list_accent_sample_duration[accent]))
    stats += '\nall selections: ' + str(list_total_selection)
    
    with open(output_dir + '/stats.txt', 'w') as f:
        f.write(stats)

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import json, os, torch, statistics, glob, librosa, pickle, torchaudio
from tqdm import tqdm
import numpy as np
import torchaudio.functional as F
import torchaudio.transforms as T

mfcc_transform = T.MFCC(
    sample_rate=22050,
    n_mfcc=39,
    melkwargs={
      'n_fft': 2048,
      'n_mels': 256,
      'hop_length': 512,
      'mel_scale': 'htk',
    }
)
base_dir = './'

In [ ]:
def extract_features(file_list, file_dir):
    file_type = file_dir.split('/')[-1].replace('.json', '')
    feature_dir = '/'.join(file_dir.split('/')[:-1])+'/39/'
    os.makedirs(os.path.dirname(feature_dir), exist_ok=True)
    feature_file = feature_dir+file_type+'_39.file'
    with open(feature_file, 'wb') as f:
        for file in tqdm(file_list):
            waveform, sample_rate = torchaudio.load(file['audio_filepath'])
            mfcc_features = mfcc_transform(waveform).mean(2).detach().numpy()
            pickle.dump(mfcc_features, f)
    print("completed", file_dir)

In [ ]:
speakers = ['african', 'indian', 'hongkong', 'philippines', 
           'england', 'scotland', 'ireland', 'australia', 
           'canada', 'us', 
           'bermuda', 'southatlandtic', 'wales', 'malaysia']
file_types = ["test", "selection", "seed"]
for speaker in speakers:
    print(speaker)
    for file_type in file_types:
#         print(file_type)
        file_dir = f'{speaker}/{file_type}.json'
        opened_file = open(file_dir)
        json_list = [json.loads(line.strip()) for line in opened_file]
        extract_features(json_list, file_dir)
    print('\n\n')

In [ ]:
seed_file_dir = 'assamese_female_english/seed.json'
seed_file = open(seed_file_dir)
seed_list = [json.loads(line.strip()) for line in seed_file]
extract_features(seed_list, seed_file_dir)

In [ ]:
def duration(json_file):
    dur_list = [json.loads(line.strip())['duration'] for line in open(json_file)]
    print(4.92*100, "requested duration")
    print(int(4.92*len(dur_list)), "is num_selected x 4.92", f"{len(dur_list)} samples")
    print(sum(dur_list), "selected duration")

In [ ]:
duration('kannada_male_english/all/budget_200/target_20/FL2MI/39/budget_100/error_model/run_2/train.json')